In [4]:
import Levenshtein
import pandas as pd


In [5]:
def find_top_uuid_matches(df1, df2):
    """Iterate through all unmapped properties and then calculate each LD against mapped properties"""
    results_df = pd.DataFrame(columns=['Concat', 'UUID1', 'Score1', 'UUID2', 'Score2', 'UUID3', 'Score3'])

    for index1, row1 in df2.iterrows():
        concat_str1 = row1['Concat']
        distances = {}

        for _, row2 in df1.iterrows():  
            concat_str2 = row2['Concat']
            distance = Levenshtein.distance(concat_str1, concat_str2)
            # set threshold to LD < 50
            if distance < 50:
                distances[row2['UUID']] = distance 
        # grab best 3 scores
        smallest_items = sorted(distances.items(), key=lambda x: x[1])[:3]  
        row_data = [concat_str1]
        
        # add to new data frame
        for i, (uuid, score) in enumerate(smallest_items):
            row_data.extend([uuid, score])

        row_data.extend(['0'] * (7 - len(row_data))) 

        results_df.loc[index1] = row_data

    return results_df


In [6]:

df1 = pd.read_csv('output/orig/School_Board.csv')
df2 = pd.read_csv('output/unmapped/2019_School_Board_unmapped.csv')
find_top_uuid_matches(df1, df2).to_csv('output/calculated/School_Board.csv', index=False)
